In [10]:
import numpy as np
import pandas as pd
from fyers_apiv3 import fyersModel
from config import CLIENT_ID, AUTH_CODE, REDIRECT_URI, RESPONSE_TYPE, STATE, SECRET_KEY, GRANT_TYPE, get_logger

# --- Exchange auth_code for access_token ---
session = fyersModel.SessionModel(
    client_id=CLIENT_ID,
    secret_key=SECRET_KEY,
    redirect_uri=REDIRECT_URI,
    response_type=RESPONSE_TYPE,
    state=STATE,
    grant_type=GRANT_TYPE
)

session.set_token(AUTH_CODE)
response = session.generate_token()

access_token = response.get("access_token")
print(access_token)
if not access_token:
    raise SystemExit("Exiting: Could not retrieve access_token")

# --- Initialize Fyers client ---
fyers = fyersModel.FyersModel(
    token=access_token,
    is_async=False,
    client_id=CLIENT_ID
)

# --- Example: Historical data for backtesting ---
data = {
    "symbol": "NSE:GOLDSTAR-SM",
    "resolution": "1D",       
    "date_format": "1",       # YYYY-MM-DD
    "range_from": "2024-01-01",
    "range_to": "2024-03-31",
    "cont_flag": "1"
}

history = fyers.history(data=data)

# --- Convert to DataFrame ---
import pandas as pd
from zoneinfo import ZoneInfo  

if history.get("s") == "ok":
    print("History data found.")
    candles = history["candles"]
    
    # Create DataFrame
    df = pd.DataFrame(candles, columns=["date", "open", "high", "low", "close", "volume"])
    df["date"] = pd.to_datetime(df["date"], unit="s", utc=True)
    df["date"] = df["date"].dt.tz_convert(ZoneInfo("Asia/Kolkata"))
    df.set_index("date", inplace=True)
    
    # Reorder columns to OCLHV format
    df = df[["open", "close", "low", "high", "volume"]]
else:
    print("Error fetching history:", history)


eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb3dHSmptTFd6YVEyai1ncC01ZlZwbklSTXNYdVV6REc3ZXpKRG41NFZHQnhybjh3SlZXdWJwNE1fUFBKQmlBUkNHZjNadi1jOUloakRlUWptWVZmNUZ6V1ZHRVRKMWN4OWp1X2xkeHl6NVdwWXBvMD0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIyMmE1NzFkM2Y3Njg4NGY4YjcyNjBjMDZkYjdjMzAyZTc5YzYyMDM3MjljOTA3Y2Y4N2RhZDM3YyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiRkFDNjIzNDQiLCJhcHBUeXBlIjoxMDAsImV4cCI6MTc1NzQ2NDIwMCwiaWF0IjoxNzU3NDM4NTYzLCJpc3MiOiJhcGkuZnllcnMuaW4iLCJuYmYiOjE3NTc0Mzg1NjMsInN1YiI6ImFjY2Vzc190b2tlbiJ9.L7UZ9TQbpQC94hj1N5xgU-nHKm7i5O8oLqKJ6215ps8
History data found.


In [11]:
df.head(), df.shape

(                            open  close    low   high  volume
 date                                                         
 2024-01-01 05:30:00+05:30  16.80  17.10  16.00  17.20  225000
 2024-01-02 05:30:00+05:30  17.10  16.90  16.40  17.10  135000
 2024-01-03 05:30:00+05:30  16.90  16.95  16.15  17.25  157500
 2024-01-04 05:30:00+05:30  16.15  16.35  16.15  16.75  247500
 2024-01-05 05:30:00+05:30  16.35  16.90  15.90  17.00  180000,
 (60, 5))

In [14]:
import plotly.graph_objects as go
import plotly.io as pio
from indicators import SMA  # your SMA function

# Open plots in browser
pio.renderers.default = "browser"

# Create figure
fig = go.Figure()

# Plot close price
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['close'],
    mode='lines',
    name='Close Price',
    line=dict(color='royalblue', width=2)
))

# Plot SMA 4
fig.add_trace(go.Scatter(
    x=df.index,
    y=SMA(df, 4),
    mode='lines',
    name='SMA 4',
    line=dict(color='orange', width=2)
))

# Plot SMA 15
fig.add_trace(go.Scatter(
    x=df.index,
    y=SMA(df, 15),
    mode='lines',
    name='SMA 15',
    line=dict(color='green', width=2)
))

# Update layout
fig.update_layout(
    title='Close Price with SMA 4 & SMA 15',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark',
    hovermode='x unified'
)

# Show interactive plot in browser
fig.show()
